In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [4]:
df = pd.read_csv('../data/bodyPerformance.csv')
print(df.head())

    age gender  height_cm  weight_kg  body fat_%  diastolic  systolic  \
0  27.0      M      172.3      75.24        21.3       80.0     130.0   
1  25.0      M      165.0      55.80        15.7       77.0     126.0   
2  31.0      M      179.6      78.00        20.1       92.0     152.0   
3  32.0      M      174.5      71.10        18.4       76.0     147.0   
4  28.0      M      173.8      67.70        17.1       70.0     127.0   

   gripForce  sit and bend forward_cm  sit-ups counts  broad jump_cm class  
0       54.9                     18.4            60.0          217.0     C  
1       36.4                     16.3            53.0          229.0     A  
2       44.8                     12.0            49.0          181.0     C  
3       41.4                     15.2            53.0          219.0     B  
4       43.5                     27.1            45.0          217.0     B  


In [6]:
df['BMI'] = df['weight_kg'] / (df['height_cm']/100) ** 2
df['bodyFatRatio'] = df['body fat_%'] / df['BMI']
df['strength_ration'] = df['gripForce'] / df['weight_kg']
df['jump_ratio'] = df['broad jump_cm'] / df['height_cm']
df['situp_ratio'] = df['sit-ups counts'] / df['age']
df['flexibility_ratio'] = df['sit and bend forward_cm'] / df['height_cm']
df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class,BMI,bodyFatRatio,strength_ration,jump_ratio,situp_ratio,flexibility_ratio
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,25.344179,0.840430,0.729665,1.259431,2.222222,0.106790
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,20.495868,0.766008,0.652330,1.387879,2.120000,0.098788
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,24.181428,0.831216,0.574359,1.007795,1.580645,0.066815
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,23.349562,0.788023,0.582278,1.255014,1.656250,0.087106
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,22.412439,0.762969,0.642541,1.248562,1.607143,0.155926


In [15]:
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df['BMI'] = pd.to_numeric(df['BMI'], errors='coerce')
df = df.dropna().reset_index(drop=True)
cols_to_check = df.columns.drop(['gender', 'class'])
df = df[(df[cols_to_check] > 0).all(axis=1)].reset_index(drop=True)
df.head(10)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class,BMI,bodyFatRatio,strength_ration,jump_ratio,situp_ratio,flexibility_ratio
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,25.344179,0.840430,0.729665,1.259431,2.222222,0.106790
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,20.495868,0.766008,0.652330,1.387879,2.120000,0.098788
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,24.181428,0.831216,0.574359,1.007795,1.580645,0.066815
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,23.349562,0.788023,0.582278,1.255014,1.656250,0.087106
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,22.412439,0.762969,0.642541,1.248562,1.607143,0.155926
5,36.0,0,165.4,55.40,22.0,64.0,119.0,23.8,21.0,27.0,153.0,B,20.250640,1.086385,0.429603,0.925030,0.750000,0.126965
6,42.0,0,164.5,63.70,32.2,72.0,135.0,22.7,0.8,18.0,146.0,D,23.540063,1.367881,0.356358,0.887538,0.428571,0.004863
7,33.0,1,174.9,77.20,36.9,84.0,137.0,45.9,12.3,42.0,234.0,B,25.236997,1.462139,0.594560,1.337907,1.272727,0.070326
8,54.0,1,166.8,67.50,27.6,85.0,165.0,40.4,18.6,34.0,148.0,C,24.261167,1.137620,0.598519,0.887290,0.629630,0.111511
9,28.0,1,185.0,84.60,14.4,81.0,156.0,57.9,12.1,55.0,213.0,B,24.718773,0.582553,0.684397,1.151351,1.964286,0.065405


In [16]:
X = df.drop('class', axis=1)
y = df['class']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
lr_model = LogisticRegression(max_iter=500)
lr_model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=500)

In [21]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [22]:
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train) 

RandomForestClassifier(n_estimators=200, random_state=42)

In [24]:
def evaluate(model, X_test_scaled, y_test):
    preds = model.predict(X_test_scaled)
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))
    print(confusion_matrix(y_test, preds))

In [25]:
print("Logistic Regression")
evaluate(lr_model, X_test_scaled, y_test)

print("KNN")
evaluate(knn_model, X_test_scaled, y_test)

print("Random Forest")
evaluate(rf_model, X_test, y_test)

Logistic Regression
Accuracy: 0.6039603960396039
              precision    recall  f1-score   support

           A       0.68      0.75      0.71       670
           B       0.46      0.44      0.45       669
           C       0.56      0.56      0.56       669
           D       0.75      0.68      0.71       517

    accuracy                           0.60      2525
   macro avg       0.61      0.61      0.61      2525
weighted avg       0.60      0.60      0.60      2525

[[503 158   9   0]
 [175 297 177  20]
 [ 49 151 373  96]
 [ 10  46 109 352]]
KNN


c:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Accuracy: 0.5837623762376237
              precision    recall  f1-score   support

           A       0.59      0.83      0.69       670
           B       0.44      0.46      0.45       669
           C       0.59      0.48      0.53       669
           D       0.84      0.56      0.67       517

    accuracy                           0.58      2525
   macro avg       0.62      0.58      0.59      2525
weighted avg       0.60      0.58      0.58      2525

[[554 103  11   2]
 [268 311  74  16]
 [ 93 220 321  35]
 [ 22  70 137 288]]
Random Forest
Accuracy: 0.7164356435643564
              precision    recall  f1-score   support

           A       0.72      0.88      0.79       670
           B       0.61      0.61      0.61       669
           C       0.71      0.65      0.68       669
           D       0.89      0.73      0.80       517

    accuracy                           0.72      2525
   macro avg       0.73      0.72      0.72      2525
weighted avg       0.72      0.72   

In [26]:
importances = rf_model.feature_importances_
feature_names = X.columns
sorted_idx = np.argsort(importances)[::-1]

print("Feature Importance Ranking:\n")

for i in sorted_idx:
    print(f"{feature_names[i]} : {importances[i]:.4f}")

Feature Importance Ranking:

sit and bend forward_cm : 0.1272
flexibility_ratio : 0.1210
sit-ups counts : 0.0964
strength_ration : 0.0732
age : 0.0640
body fat_% : 0.0599
BMI : 0.0598
situp_ratio : 0.0571
jump_ratio : 0.0473
weight_kg : 0.0471
bodyFatRatio : 0.0460
gripForce : 0.0452
broad jump_cm : 0.0420
height_cm : 0.0391
systolic : 0.0335
diastolic : 0.0313
gender : 0.0100


Tuned RF accuracy: 0.714059405940594
